In [11]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Load dataset
measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:8000]
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)

# Extract features for training data
alice_train_CIRs = trainCIR[:, alice_channel, :, :]  # (6400, 251, 2)
eve_train_CIRs = trainCIR[:, eve_channel, :, :]  # (6400, 251, 2)
train_cirs = np.vstack((alice_train_CIRs, eve_train_CIRs))

# Apply PCA on the second dimension of each CIR sample
n_components = 2
train_cirs_pca = []

for cir in train_cirs:
    scaler = StandardScaler()
    cir_scaled = scaler.fit_transform(cir)  # Shape: (251, 2)
    # print(f'cir_scaled : {cir_scaled.T.shape}')
    pca = PCA(n_components=n_components)
    cir_pca_transformed = pca.fit_transform(cir_scaled.T).T  # Shape: (251, n_components)
    
    # print(cir_pca_transformed.shape)
    # print()
    train_cirs_pca.append(cir_pca_transformed)  # Transpose to get (n_components, 251)

train_cirs_pca = np.array(train_cirs_pca)
print(f'train_cirs_pca : {train_cirs_pca.shape}')

# Create labels for training data
alice_train_labels = np.zeros(alice_train_CIRs.shape[0])  # Label '0' for Alice.
eve_train_labels = np.ones(eve_train_CIRs.shape[0])  # Label '1' for Eve.
train_labels = np.hstack((alice_train_labels, eve_train_labels))

# Reshape train data for classification
train_features = train_cirs_pca.reshape(train_cirs_pca.shape[0], -1)

# Extract features for test data
alice_test_CIRs = testCIR[:, alice_channel, :, :]
eve_test_CIRs = testCIR[:, eve_channel, :, :]
test_cirs = np.vstack((alice_test_CIRs, eve_test_CIRs))

test_cirs_pca = []
for cir in test_cirs:
    scaler = StandardScaler()
    cir_scaled = scaler.fit_transform(cir)  # Shape: (251, 2)
    
    pca = PCA(n_components=n_components)
    cir_pca_transformed = pca.fit_transform(cir_scaled.T)  # Shape: (251, n_components)
    
    test_cirs_pca.append(cir_pca_transformed.T)  # Transpose to get (n_components, 251)

test_cirs_pca = np.array(test_cirs_pca)

# Create labels for test data
alice_test_labels = np.zeros(alice_test_CIRs.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(eve_test_CIRs.shape[0])  # Label '1' for Eve.
test_labels = np.hstack((alice_test_labels, eve_test_labels))

# Reshape test data for classification
test_features = test_cirs_pca.reshape(test_cirs_pca.shape[0], -1)


trainData - (6400, 15, 251, 2)
testData - (1600, 15, 251, 2)
train_cirs_pca : (12800, 2, 2)


In [12]:
# Step 5: Classification using K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_features, train_labels)

# Step 6: Classifying Test Data and Evaluating the Model
predictions = knn.predict(test_features)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# Gamma calculation
gamma = (tp + fn) / (tn + fp)

# Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

Classification Accuracy: 51.25%
tp: 806
tn: 834
fp: 766
fn: 794
MDR: 0.47875
FAR: 0.49625
AR: 0.5125
